In [1]:
path = './ntust-ir-2020_hw5_new/'
# path = '../HW2-BM25/ntust-ir-2020/'
import numpy as np
import numba as nb
import time
import math
from tqdm import tqdm
from math import log
from collections import Counter
from numba.typed import List
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

In [2]:
# data of doc and query
doc = []
query = []
# length of each doc
doc_len = []
# Name of doc and query
doc_list = []
query_list = []
# TF of doc and query
tf_doc = []
tf_q = []
# BM25 query vocabulary
bm25_vocabulary = set()
# BackGround voc
bg_counter = Counter()
# -----------------------------------------------------------------------------
# get doc files and query files
with open(path + 'query_list.txt', 'r') as fq:
    line = fq.read().splitlines()
    for l in line:
        query_list.append(l)
        f_temp = open(path + 'queries/{}.txt'.format(l))
        temp_query = f_temp.read().split()
        tf_q.append(Counter(temp_query))
        for w in temp_query: bm25_vocabulary.add(w)
        query.append(temp_query)
        f_temp.close()
with open(path + 'doc_list.txt', 'r') as fd:
    line = fd.read().splitlines()
    for l in tqdm(line):
        doc_list.append(l)
        f_temp = open(path + 'docs/{}.txt'.format(l))
        temp_doc = f_temp.read().split()
        tf_doc.append(Counter(temp_doc))
        bg_counter.update(temp_doc)
        doc.append(temp_doc)
        doc_len.append(len(temp_doc))
        f_temp.close()

100%|██████████| 30000/30000 [00:06<00:00, 4385.97it/s]


In [3]:
# vsm vocabulary
vsm_voc = set()
vsm_voc = bm25_vocabulary
bm25_vocabulary = list(bm25_vocabulary)
# All doc, query and vocabulary length
d_len = len(doc)
q_len = len(query)
bm_w_len = len(bm25_vocabulary)

In [4]:
# Build VSM vocabulary
for k,v in bg_counter.items():
    if(v > 30): vsm_voc.add(k)
vsm_voc = list(vsm_voc)
w_len = len(vsm_voc)

In [35]:
bg_len = sum(doc_len)

In [5]:
w_len

16740

In [36]:
# BackGround Model
P_bg = dict()
for k,v in bg_counter.items():
    P_bg[k] = v / bg_len

In [6]:
# Build TF_key and TF_val
TF_doc_k = List()
TF_doc_v = List()
for d in tqdm(range(d_len)):
    TF_doc_k.append(List([k for k in tf_doc[d].keys()]))
    TF_doc_v.append(List([v for v in tf_doc[d].values()]))
TF_q_k = List()
TF_q_v = List()
for q in range(q_len):
    TF_q_k.append(List([k for k in tf_q[q].keys()]))
    TF_q_v.append(List([v for v in tf_q[q].values()]))

100%|██████████| 30000/30000 [00:29<00:00, 1032.12it/s]


In [7]:
# BM25 IDF
@nb.njit
def bm_IDF(idf, tf_key, bm25_voc, dl, bm25l):
    for d in range(dl):
        for v in range(bm25l):
            if(bm25_voc[v] in tf_key[d]):
                idf[v] += 1 
    return idf

In [8]:
# BM25 IDF
bm_idf = np.zeros(bm_w_len)
time_idf = time.time()
bm_idf = bm_IDF(bm_idf, TF_doc_k, bm25_vocabulary, d_len, bm_w_len)
print('time: {:.5f} mins\n'.format((time.time() - time_idf) / 60))

time: 1.27932 mins



In [9]:
# B25 Model
@nb.njit
def BM25(sim, k1, b_, N_, avg, dl_list, dl, ql, tf_key, tf_val, tf_q_key, idf_, bm_voc):
    for q in range(ql):
        for d in range(dl):
            sum = 0.0
            for w in tf_q_key[q]:
                sum_tmp = 1.0
                if(w in tf_key[d]):
                    w_tf = tf_val[d][tf_key[d].index(w)]
                    ni_loc = bm_voc.index(w)
                    ni = idf_[ni_loc]
                    sum_tmp = (k1 + 1) * w_tf / (k1 * ((1 - b) + b * dl_list[d] / avg) + w_tf)
                    sum_tmp *= log((N_ - ni + 0.5) / (ni + 0.5))
                sum += sum_tmp
            sim[q][d] = sum
    return sim

In [10]:
# BM25 model variable
bm_sim = np.zeros([q_len, d_len], dtype=float)
K1 = 0.8
b = 0.7
N = d_len
avg_dl = np.average(doc_len)

In [11]:
time_bm = time.time()
bm_sim = BM25(bm_sim, K1, b, N, avg_dl, doc_len, d_len, q_len, TF_doc_k, TF_doc_v, TF_q_k, bm_idf, bm25_vocabulary)
print('time: {:.2f} mins\n'.format((time.time() - time_bm) / 60))

time: 1.90 mins



In [12]:
np.save('bm.npy', bm_sim)

In [13]:
# VSM IDF
@nb.njit
def IDF(idf, tf_key, voc, dl, wl):
    for d in range(dl):
        for v in range(wl):
            if(voc[v] in tf_key[d]):
                idf[v] += 1
    for v in range(wl):
        if(idf[v] != 0):
            idf[v] = log(dl / idf[v])
    return idf

In [14]:
# Build VSM IDF
vsm_doc_idf = np.zeros(w_len)
vsm_d_time = time.time()
vsm_doc_idf = IDF(vsm_doc_idf, TF_doc_k, vsm_voc, d_len, w_len)
print('time: {:.2f} mins\n'.format((time.time() - vsm_d_time) / 60))

time: 65.89 mins



In [15]:
np.save('doc_idf.npy', vsm_doc_idf)

In [16]:
@nb.njit
def TF_IDF(tf_idf, idf, tf_d_k, tf_d_v, voc, dl, wl):
    for w in range(wl):
        idf_ = idf[w]
        for d in range(dl):
            if(voc[w] in tf_d_k[d]):
                loc = tf_d_k[d].index(voc[w])
                tf_idf[d][w] = idf_ * tf_d_v[d][loc]
    return tf_idf

In [17]:
# Build VSM TF-IDF
vsm_doc_tfidf = np.zeros([d_len, w_len])
vsm_tfidf_time = time.time()
vsm_doc_tfidf = TF_IDF(vsm_doc_tfidf, vsm_doc_idf, TF_doc_k, TF_doc_v, vsm_voc, d_len, w_len)
print('time: {:.2f} mins\n'.format((time.time() - vsm_tfidf_time) / 60))

time: 91.86 mins



In [18]:
np.save('vsm.npy', vsm_doc_tfidf)

In [19]:
# Query TF-IDF
@nb.njit
def Q_TF_IDF(tf_idf, idf, tf_q_k, tf_q_v, voc, ql, wl):
    for w in range(wl):
        idf_ = idf[w]
        for q in range(ql):
            if(voc[w] in tf_q_k[q]):
                loc = tf_q_k[q].index(voc[w])
                tf_idf[q][w] = idf_ * (0.5 + 0.5 * tf_q_v[q][loc])
    return tf_idf

In [20]:
# Build Query TF-IDF
vsm_query_tfidf = np.zeros([q_len, w_len])
vsm_query_tfidf = Q_TF_IDF(vsm_query_tfidf, vsm_doc_idf, TF_q_k, TF_q_v, vsm_voc, q_len, w_len)

In [22]:
# Cos Sim
vsm_cos_sim = np.zeros([q_len, d_len], dtype=float)
for q in tqdm(range(q_len)):
    for d in range(d_len):
        vsm_cos_sim[q][d] = 1 - spatial.distance.cosine(vsm_query_tfidf[q], vsm_doc_tfidf[d])

100%|██████████| 150/150 [05:02<00:00,  2.02s/it]


In [37]:
# Get relevance document
Rele_len = int(3)
Rq = np.zeros([q_len, Rele_len], dtype=np.int64)
for q in tqdm(range(q_len)):
    arg = np.argsort(-bm_sim[q])[:Rele_len]
    for r in range(Rele_len):
        Rq[q][r] = arg[r]

100%|██████████| 150/150 [00:00<00:00, 1690.84it/s]


In [38]:
# New Query
def N_query(new_q, old_q, rel_doc, rel_len, doc_tfidf, ql, wl, alpha_, beta_, iter_):
    for i in tqdm(range(iter_)):
        for q in range(ql):
                s1 = old_q[q] * alpha_
                s2 = beta_ / rel_len
                for d in range(rel_len):
                    s1 = np.vstack((s1, np.array((doc_tfidf[rel_doc[q][d]] * s2))))
                new_q[q] = np.sum(s1, axis=0)
                for k in range(wl):
                    if(new_q[q][k] < 0):
                        new_q[q][k] = 0
    return new_q

In [39]:
# new query model
alpha = 1
beta = 0.75
new_query = np.zeros([q_len, w_len])
new_query = N_query(new_query, vsm_query_tfidf, Rq, Rele_len, vsm_doc_tfidf, q_len, w_len, alpha, beta, 5)

100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


In [40]:
# Cos Sim
cos_sim = np.zeros([q_len, d_len], dtype=float)
for q in tqdm(range(q_len)):
    for d in range(d_len):
        cos_sim[q][d] = 1 - spatial.distance.cosine(new_query[q], vsm_doc_tfidf[d])

100%|██████████| 150/150 [05:05<00:00,  2.03s/it]


In [41]:
# Create output file
fp = open("roicchio_bm_{}_{}.txt".format(alpha, beta), "w")
fp.write("Query,RetrievedDocuments\n")
for i in range(q_len):
    fp.write('{},'.format(query_list[i]))
    for s in np.argsort(-cos_sim[i])[:5000]:
        fp.write(doc_list[s] + ' ')
    fp.write('\n')
fp.close()